In [1]:
import yaml
import os
os.chdir('..')
print(os.getcwd())

z:\OMOP\omop_etl


In [2]:
%run -i run_etl.py

Preloading drug_exposure (order) ...
preload complete. Elapsed time 00:00:06
Preloading drug_exposure (admin) ...
preload complete. Elapsed time 00:00:06
Loading drug_exposure ...
load_table complete. Elapsed time 00:00:13
5904 rows affected
z:\OMOP\omop_etl


In [3]:
from omop_etl.postproc.load import Loader, truncate
load = Loader('config.yml')
# load.load_person()
# load.preload('measurement','height')

In [5]:
load.load_table('condition_occurrence')

Preloading condition_occurrence (None) ...
preload complete. Elapsed time 00:00:02
Loading condition_occurrence ...
load_table complete. Elapsed time 00:00:03


'1322 rows affected'

In [3]:
# truncate('preload','drug_exposure', load.engine)

In [2]:
f = open('config.yml')
load = yaml.safe_load(f)
# print(preload)
# list(preload['preload']['drug'].keys())

In [4]:
STAGE = {
    'person': 'PERSON',
    'death': 'DEATH',
    'condition_occurrence': 'CONDITION',
    'procedure_occurrence': {
        'cpt': 'PROCEDURE_CPT',
        'icd': 'PROCEDURE_ICD'
    },
    'drug_exposure': {
        'order': 'DRUG_ADMIN', 
        'admin': 'DRUG_ORDER'
    },
    'measurement': {
        'bp': 'MEASUREMENT_BP',
        'res_pip': 'MEASUREMENT_Res_PIP',
        'heart_rate': 'MEASUREMENT_HeartRate',
        'height': 'MEASUREMENT_Height'
    }
}

dp_list = []
for t in load['load'].keys():
    if load['load'][t]:
        for part in load['load'][t].keys(): 
            dp_list.append(STAGE[t][part])
    else: 
        dp_list.append(STAGE[t])
        
print(dp_list)

['PERSON', 'DEATH', 'CONDITION', 'PROCEDURE_ICD', 'PROCEDURE_CPT', 'DRUG_ADMIN', 'DRUG_ORDER', 'MEASUREMENT_BP', 'MEASUREMENT_HeartRate', 'MEASUREMENT_Height']


In [6]:
from omop_etl.datastore import DataStore, execute
omop = DataStore('config.yml')
# t = omop.list_tables(in_schema=['preload'])

In [8]:
omop.config_param['load']['condition_occurrence']

In [21]:
# for table in t.Table:
#     execute(f'drop table if exists preload.{table}', omop.engine)

In [2]:
%run -i omop_etl/pullrawdata/pull_raw_data_v2.py

Start time: 2021-01-14 09:51:51.962589

